<a href="https://colab.research.google.com/github/popo169/Portfolio-Project/blob/main/Pizza_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
Business intelligence is very important nowadays. It helps you to know better of your company including how the sales recently, which item is going to out of stock and most importantly, what is the profit.

In this project, I am going to demonstrate how to use data from a pizza shop along with SQL to create 3 dashboards for business intelligence.





##Data content
There are 10 tables in the dataset.

Table name | Content | Columns
--- | --- | ---
Address | Customers' address and zip code | 'add_id', 'delivery_address1', 'delivery_address2', 'delivery_city', 'delivery_zipcode'
Customers | First and last name | 'cust_id', 'cust_firstname', 'cust_lastname'
Ingredient | Ingredient name, weight, unit and price | 'ing_id', 'ing_name', 'ing_weight', 'ing_meas', 'ing_price'
Inventory | Quantity of inventory | 'inv_id', 'item_id', 'quantity'
Item | Item name, code, category, size and price | 'item_id', 'sku', 'item_name', 'item_cat', 'item_size', 'item_price'
Orders | Order create time, item ordered, quantity, customer ID, delivery/pick up, address ID | 'row_id', 'order_id', 'created_at', 'item_id', 'quantity', 'cust_id', 'delivery', 'add_id'
Recipe | Recipe ID, ingredient ID, required quantity | 'row_id', 'recipe_id', 'ing_id', 'quantity'
Rota | Duty date, shift ID, staff ID | 'row_id', 'rota_id', 'date', 'shift_id', 'staff_id'
Shift | Day of week, start time and end time | 'shift_id', 'day_of_week', 'start_time', 'end_time'
Staff | Staff first and last name, position, hourly rate | 'staff_id', 'first_name', 'last_name', 'position', 'hourly_rate'

##Dashboard 1
This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##Dashboard 2
The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##Dashboard 3
The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

##Import data

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import library
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# define functions

def pd_to_sqlDB(input_df: pd.DataFrame, table_name: str, db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 2: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()

    # Step 3: Create Table
    sql_string = f"""CREATE TABLE IF NOT EXISTS {table_name} ({cols_string});"""
    cur.execute(sql_string)

    # Step 4: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
  
    # Step 5: Commit the changes and close the connection
    con.commit()
    con.close()



def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [ ]:
# import data
address=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/address.csv')
customers=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/customers.csv')
ingredient=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/ingredient.csv')
inventory=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/inventory.csv')
item=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/item.csv')
orders=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/orders.csv')
recipe=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/recipe.csv')
rota=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/rota.csv')
shift=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/shift.csv')
staff=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/staff.csv')

In [ ]:
# import data to sql tables
pd_to_sqlDB(address,"address","pizza_db")
pd_to_sqlDB(customers,"customers","pizza_db")
pd_to_sqlDB(ingredient,"ingredient","pizza_db")
pd_to_sqlDB(inventory,"inventory","pizza_db")
pd_to_sqlDB(item,"item","pizza_db")
pd_to_sqlDB(orders,"orders","pizza_db")
pd_to_sqlDB(recipe,"recipe","pizza_db")
pd_to_sqlDB(rota,"rota","pizza_db")
pd_to_sqlDB(shift,"shift","pizza_db")
pd_to_sqlDB(staff,"staff","pizza_db")

#Query for 1st dashboard

In [ ]:
# Query table for the 1st dashboard
query="""
SELECT
  o.order_id,
  o.quantity*i.item_price AS order_sales,
  o.quantity,
  i.item_cat,
  i.item_name,
  o.created_at,
  a.delivery_address1,
  -- a.delivery_address2, --no data in address 2
  a.delivery_city,
  a.delivery_zipcode,
  o.delivery
FROM orders o
LEFT JOIN item i ON o.item_id=i.item_id
LEFT JOIN address a ON o.add_id=a.add_id;
"""
result1=sql_query_to_pd(query,"pizza_db")

In [ ]:
# Convert to appropriate data type
result1['created_at']=pd.to_datetime(result1['created_at'],format="%d/%m/%Y %H:%M")
result1['delivery_zipcode']=result1['delivery_zipcode'].astype('str')

In [ ]:
# Table for 1st dashboard 
result1

,order_id,order_sales,quantity,item_cat,item_name,created_at,delivery_address1,delivery_city,delivery_zipcode,delivery
0,109,24,2,Pizza,Pizza Margherita Reg,2022-08-10 13:22:00,607 Trails End Road,Manchester,6042,1
1,110,16,1,Pizza,Pizza Diavola (hot) Reg,2022-08-10 13:53:00,25 Cliffside Drive,Manchester,6042,1
2,111,12,1,Pizza,Pizza Margherita Reg,2022-08-10 13:32:00,56 Concord Road,Manchester,6042,1
3,111,16,1,Pizza,Pizza Diavola (hot) Reg,2022-08-10 13:32:00,56 Concord Road,Manchester,6042,1
4,112,57,3,Pizza,Pizza Quattro Formaggi Large,2022-08-10 19:19:00,82 Lookout Mountain Drive,Manchester,6040,0
...,...,...,...,...,...,...,...,...,...,...
309,165,6,1,Drink,Coca Cola Diet 1.5l,2022-08-10 19:19:00,35 Lakewood Circle South,Manchester,6040,0
310,166,16,1,Pizza,Pizza Quattro Formaggi Reg,2022-08-10 14:22:00,91 Eldridge Street,Manchester,6040,1
311,166,5,1,Side,Breadsticks,2022-08-10 14:22:00,91 Eldridge Street,Manchester,6040,1
312,166,5,1,Dessert,Chocolate Brownie,2022-08-10 14:22:00,91 Eldridge Street,Manchester,6040,1


##Dashboard 1
This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning of losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##1. Total orders

In [ ]:
ttl_no_order=len(result1['order_id'].unique())
print(f'The total number of order is {ttl_no_order}')

The total number of order is 58


##2. Total sales

In [ ]:
ttl_sales=sum(result1['order_sales'])
print(f'The total sales is ${ttl_sales}')

The total sales is $5967


##3. Total item sold

In [ ]:
ttl_item_sold=sum(result1['quantity'])
print(f'The total item sold is {ttl_item_sold}')

The total item sold is 626


##4. Average order value

In [ ]:
avg_order_val=result1.groupby(['order_id'])['order_sales'].sum().mean()
print(f'The average order value is ${avg_order_val:.2f}')

The average order value is $102.88


##5. Sales by category

In [ ]:
sales_by_cat=result1.groupby(['item_cat'])['order_sales'].sum().to_frame().sort_values('order_sales',ascending=False)
print(f'The average order value is \n',sales_by_cat)

The average order value is 
           order_sales
item_cat             
Pizza            3816
Dessert           915
Side              801
Drink             435


##6. Top selling item

In [ ]:
top_item=result1.groupby(['item_name'])['order_sales'].sum().idxmax()
print(f'The top selling item is {top_item}')

The top selling item is Pizza Quattro Formaggi Large


##7. Order by hour

In [ ]:
order_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_id'].count().to_frame('count').rename_axis('hour')
print(f'The order by hour is\n',order_by_hour)

The order by hour is
       count
hour       
12       30
13       96
14       11
18       15
19       67
20       37
21       28
22       30


##8. Sales by hour

In [ ]:
sales_by_hour=result1.groupby(result1['created_at'].dt.hour)['order_sales'].sum().to_frame('sales').rename_axis('hour')
print(f'The sales by hour is\n',sales_by_hour)

The sales by hour is
       sales
hour       
12      487
13     1399
14       91
18      419
19     1552
20      567
21      470
22      982


##9. Order by address

In [ ]:
order_by_address=result1['delivery_address1'].value_counts().to_frame('count').rename_axis('address')
print(f'The order by address is\n',order_by_address)

The order by address is
                             count
address                          
150 Carter Street              17
68 Princeton Street            12
18 Cambridge Street            12
125 Summer Street              11
425 Middle Turnpike East        9
95 Briarwood Drive              9
60 Desousa Drive                9
61 Plymouth Lane                9
697 Parker Street               9
29 Lucian Street                8
89 High Ledge Circle            7
44 Downey Drive                 7
123 Elizabeth Drive             7
211 Oak Street                  7
145 Saint John Street           7
86 Highland Street              6
22 Star Farms Drive             6
61 Hills Street                 6
225 Kennedy Road                6
184 Woodland Street             6
126 Garth Road                  6
65 Arcellia Drive               6
34 Holyoke Road                 5
4 Orchard Street                5
25 Edwards Street               5
117 Adelaide Road               5
310 Timrod Road        

##10. Order by delivery/pick up

In [ ]:
order_by_delivery=result1['delivery'].value_counts().to_frame('count').rename_axis('delivery(1)/pick up(0)')
print(f'The order by delivery/pick up is\n',order_by_delivery)

The order by delivery/pick up is
                         count
delivery(1)/pick up(0)       
1                         239
0                          75


#Query for 2nd dashboard

In [ ]:
# Query table for the 2nd dashboard
query="""
SELECT
  sub1.item_name,
  sub1.ing_id,
  sub1.ing_name,
  sub1.ing_weight,
  sub1.ing_price,
  sub1.order_quan,
  sub1.rep_quan,
  sub1.order_quan*sub1.rep_quan AS quantity_required,
  sub1.ing_price/sub1.ing_weight AS unit_cost,
  (sub1.order_quan*sub1.rep_quan)*(sub1.ing_price/sub1.ing_weight) AS ingredient_cost
FROM
    (SELECT
      o.item_id,
      rep.recipe_id,
      i.item_name,
      rep.ing_id,
      ing.ing_name,
      SUM(o.quantity)  AS order_quan,
      rep.quantity AS rep_quan,
      ing.ing_price,
      ing.ing_weight
    FROM orders o
    LEFT JOIN item i ON o.item_id=i.item_id
    LEFT JOIN recipe rep ON i.sku=rep.recipe_id
    LEFT JOIN ingredient ing ON rep.ing_id=ing.ing_id
    GROUP BY
      o.item_id, 
      rep.recipe_id, 
      i.item_name,
      rep.ing_id,
      rep.quantity,
      ing.ing_name,
      ing.ing_weight,
      ing.ing_price) sub1;
"""
result2=sql_query_to_pd(query,"pizza_db")
result2['item_cost']=result2['rep_quan']*result2['unit_cost']
result2

,item_name,ing_id,ing_name,ing_weight,ing_price,order_quan,rep_quan,quantity_required,unit_cost,ingredient_cost,item_cost
0,Pizza Margherita Reg,ING001,Pizza dough ball (8 pack),2000,4.22,18,250,4500,0.002110,9.4950,0.527500
1,Pizza Margherita Reg,ING002,Tomato sauce,4500,3.89,18,80,1440,0.000864,1.2448,0.069156
2,Pizza Margherita Reg,ING003,Mozzarella cheese,2500,14.45,18,170,3060,0.005780,17.6868,0.982600
3,Pizza Margherita Reg,ING004,Dried oregano,500,5.99,18,5,90,0.011980,1.0782,0.059900
4,Pizza Margherita Large,ING001,Pizza dough ball (8 pack),2000,4.22,7,300,2100,0.002110,4.4310,0.633000
...,...,...,...,...,...,...,...,...,...,...,...
104,Fanta Diet 1.5l,ING043,Fanta Diet 1.5l,1,0.96,2,1,2,0.960000,1.9200,0.960000
105,San Pelligrino 33cl,ING044,San Pelligrino 33cl,1,0.36,4,1,4,0.360000,1.4400,0.360000
106,San Pelligrino 1.5l,ING045,San Pelligrino 1.5l,1,0.86,9,1,9,0.860000,7.7400,0.860000
107,Perrier 33cl,ING046,Perrier 33cl,1,0.36,3,1,3,0.360000,1.0800,0.360000


##Dashboard 2
The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##1. Total quantity required for orders by ingredient

In [ ]:
quantity_by_ingredient=result2.groupby('ing_name')['quantity_required'].sum().to_frame()
print(f'The total quantity required for orders by ingredient is\n',quantity_by_ingredient)

The total quantity required for orders by ingredient is
                            quantity_required
ing_name                                    
Anchovies                               1850
Banoffee pie                            4200
Caesar dressing                          740
Calamari                                1250
Capers                                   133
Chicken wings                           3600
Chilli pepper                            705
Chocolate brownie                       3225
Chocolate ice cream                     3700
Coca Cola Diet 1.5l                        1
Coca Cola Diet 33cl                        8
Coca Cola Regular 1.5l                    10
Coca Cola Regular 33cl                    14
Croutons                                1850
Dried oregano                            146
Eggplant                                2580
Fanta Diet 1.5l                            2
Fanta Regular 1.5l                         6
Fanta Regular 33cl                        2

##2. Total cost of ingredients for orders

In [ ]:
cost_by_ingredient=result2.groupby('ing_name')['ingredient_cost'].sum().to_frame()
print(f'The total cost of ingredients for orders is\n',cost_by_ingredient.round(2))

The total cost of ingredients for orders is
                            ingredient_cost
ing_name                                  
Anchovies                            20.33
Banoffee pie                          4.02
Caesar dressing                       3.50
Calamari                             14.38
Capers                                0.55
Chicken wings                        41.90
Chilli pepper                         4.58
Chocolate brownie                     5.74
Chocolate ice cream                  12.70
Coca Cola Diet 1.5l                   0.96
Coca Cola Diet 33cl                   3.28
Coca Cola Regular 1.5l                9.60
Coca Cola Regular 33cl                5.74
Croutons                              7.77
Dried oregano                         1.75
Eggplant                              4.90
Fanta Diet 1.5l                       1.92
Fanta Regular 1.5l                    5.76
Fanta Regular 33cl                   11.89
Fruit salad                           4.06
Garlic an

##3. Calculated cost of pizza

In [ ]:
cost_of_pizza=result2.groupby('item_name')[['item_cost']].sum()
filt=(cost_of_pizza.index.str.contains("Pizza"))
print(f'The cost_of_pizza is\n',cost_of_pizza[filt].round(2))

The cost_of_pizza is
                               item_cost
item_name                              
Pizza Diavola (hot) Large          2.73
Pizza Diavola (hot) Reg            2.18
Pizza Hawaiian Large               3.00
Pizza Hawaiian Reg                 2.55
Pizza Margherita Large             1.97
Pizza Margherita Reg               1.64
Pizza Napolitana Large             2.45
Pizza Napolitana Reg               2.70
Pizza Parmigiana Large             3.66
Pizza Parmigiana Reg               3.08
Pizza Pepperoni Large              4.20
Pizza Pepperoni Reg                3.51
Pizza Quattro Formaggi Large       5.13
Pizza Quattro Formaggi Reg         4.29
Pizza Seafood Large                6.13
Pizza Seafood Reg                  5.23


In [ ]:
# use previous query for following questions
pd_to_sqlDB(result2,"result2","pizza_db")

# query to answer remaining question in dashboard2
query="""
SELECT
  sub2.ing_name,
  sub2.ordered_weight,
  (ing.ing_weight*inv.quantity) AS total_inv_weight
FROM
  (SELECT
    ing_id,
    ing_name,
    SUM(quantity_required) AS ordered_weight
  FROM result2
  GROUP BY 
  ing_name, ing_id) sub2
LEFT JOIN inventory inv ON inv.item_id = sub2.ing_id
LEFT JOIN ingredient ing ON ing.ing_id = sub2.ing_id;
"""
result3=sql_query_to_pd(query,"pizza_db")
result3

,ing_name,ordered_weight,total_inv_weight
0,Anchovies,1850,2000
1,Banoffee pie,4200,2400
2,Caesar dressing,740,19000
3,Calamari,1250,7500
4,Capers,133,2000
5,Chicken wings,3600,24000
6,Chilli pepper,705,4000
7,Chocolate brownie,3225,5000
8,Chocolate ice cream,3700,9000
9,Coca Cola Diet 1.5l,1,120


##4. Percentage stock remaining by ingredient


In [ ]:
result3['perc_remain']=((result3['total_inv_weight']-result3['ordered_weight'])/result3['total_inv_weight']*100)
print(f'The percentage remaining is\n',result3[['ing_name','perc_remain']].round(2))

The percentage remaining is
                      ing_name  perc_remain
0                   Anchovies         7.50
1                Banoffee pie       -75.00
2             Caesar dressing        96.11
3                    Calamari        83.33
4                      Capers        93.35
5               Chicken wings        85.00
6               Chilli pepper        82.38
7           Chocolate brownie        35.50
8         Chocolate ice cream        58.89
9         Coca Cola Diet 1.5l        99.17
10        Coca Cola Diet 33cl        93.33
11     Coca Cola Regular 1.5l        91.67
12     Coca Cola Regular 33cl        88.33
13                   Croutons        70.40
14              Dried oregano        92.70
15                   Eggplant        87.10
16            Fanta Diet 1.5l        98.33
17         Fanta Regular 1.5l        95.00
18         Fanta Regular 33cl        75.83
19                Fruit salad        71.50
20  Garlic and parsley butter        97.17
21          Gorgonzola ch

##5. List of ingredients to re-order based on remaining inventory

In [ ]:
print(f'The ingredients need to re-order is\n',result3[['ing_name','perc_remain']].sort_values("perc_remain").round(2))

The ingredients need to re-order is
                      ing_name  perc_remain
1                Banoffee pie       -75.00
0                   Anchovies         7.50
30  Pizza dough ball (8 pack)        21.35
31             Ricotta cheese        31.20
7           Chocolate brownie        35.50
24            Parmesan cheese        44.60
37               Spicy salami        51.57
23          Mozzarella cheese        57.70
8         Chocolate ice cream        58.89
13                   Croutons        70.40
21          Gorgonzola cheese        70.51
19                Fruit salad        71.50
25                  Pepperoni        74.72
18         Fanta Regular 33cl        75.83
43                       Tuna        79.17
42              Tomato sauce         81.69
33  Rotisserie chicken pieces        82.24
6               Chilli pepper        82.38
3                    Calamari        83.33
5               Chicken wings        85.00
15                   Eggplant        87.10
36               

#Query for 3rd dashboard

In [ ]:
# Query table for the 3rd dashboard
query="""
SELECT
  st.first_name,
  st.last_name,
  st.hourly_rate,
  sh.day_of_week,
  sh.start_time,
  sh.end_time
FROM staff st
LEFT JOIN rota ro ON ro.staff_id=st.staff_id
LEFT JOIN shift sh ON sh.shift_id=ro.shift_id
"""
result4=sql_query_to_pd(query,"pizza_db")
result4['start_time']=pd.to_timedelta(result4['start_time'].str.strip())
result4['end_time']=pd.to_timedelta(result4['end_time'].str.strip())
result4['hour_worked']=(result4['end_time']-result4['start_time'])/np.timedelta64(1, 'h')
result4

,first_name,last_name,hourly_rate,day_of_week,start_time,end_time,hour_worked
0,Mindy,Sloan,17.25,Wednesday,0 days 10:30:00,0 days 14:30:00,4.0
1,Mindy,Sloan,17.25,Wednesday,0 days 18:30:00,0 days 23:00:00,4.5
2,Mindy,Sloan,17.25,Thursday,0 days 10:30:00,0 days 14:30:00,4.0
3,Mindy,Sloan,17.25,Thursday,0 days 18:30:00,0 days 23:00:00,4.5
4,Mindy,Sloan,17.25,Friday,0 days 10:30:00,0 days 14:30:00,4.0
5,Mindy,Sloan,17.25,Friday,0 days 18:30:00,0 days 23:00:00,4.5
6,Luqman,Cantu,21.50,Wednesday,0 days 10:30:00,0 days 14:30:00,4.0
7,Luqman,Cantu,21.50,Wednesday,0 days 18:30:00,0 days 23:00:00,4.5
8,Luqman,Cantu,21.50,Thursday,0 days 10:30:00,0 days 14:30:00,4.0
9,Luqman,Cantu,21.50,Thursday,0 days 18:30:00,0 days 23:00:00,4.5


##Dashboard 3
The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

##1. Hours worked by staff member

In [ ]:
hours_worked_by_staff=result4.groupby(['first_name'])['hour_worked'].sum()
print(f'The hours worked by staff is\n',hours_worked_by_staff)

The hours worked by staff is
 first_name
Amiyah         0.0
Amrit          0.0
Areeb          0.0
Arran         17.0
Desiree       25.5
Faraz          0.0
Ivan          17.0
Johnathon     17.0
Lilly-Rose    25.5
Luqman        25.5
Matilda        0.0
Mindy         25.5
Sana           0.0
Seren         17.0
Talha          0.0
Zachery        0.0
Name: hour_worked, dtype: float64


##2. Total hours worked

In [ ]:
ttl_hour_worked=hours_worked_by_staff.sum()
print(f'The total hour worked is {ttl_hour_worked}')

The total hour worked is 170.0


##3. Cost per staff member

In [ ]:
cost_tab=result4.groupby(['first_name'])[['hour_worked','hourly_rate']].agg({'hour_worked':'sum','hourly_rate':'mean'})
cost_tab['cost']=cost_tab['hour_worked'].multiply(cost_tab['hourly_rate']).round(2)
print(f'The cost per staff is \n',cost_tab)

The cost per staff is 
             hour_worked  hourly_rate    cost
first_name                                  
Amiyah              0.0        14.50    0.00
Amrit               0.0        14.50    0.00
Areeb               0.0        14.50    0.00
Arran              17.0        21.50  365.50
Desiree            25.5        14.50  369.75
Faraz               0.0        21.50    0.00
Ivan               17.0        14.50  246.50
Johnathon          17.0        14.50  246.50
Lilly-Rose         25.5        14.50  369.75
Luqman             25.5        21.50  548.25
Matilda             0.0        14.50    0.00
Mindy              25.5        17.25  439.88
Sana                0.0        21.50    0.00
Seren              17.0        17.25  293.25
Talha               0.0        17.25    0.00
Zachery             0.0        17.25    0.00


##4. Total staff cost

In [ ]:
ttl_staff_cost=cost_tab['cost'].sum()
print(f'The total staff cost is ${ttl_staff_cost}')

The total staff cost is $2879.38


#Export csv for visualization

In [ ]:
result1.to_csv('/content/gdrive/My Drive/SQL/export/pizza1.csv',index=False)
result2.to_csv('/content/gdrive/My Drive/SQL/export/pizza2.csv',index=False)
result3.to_csv('/content/gdrive/My Drive/SQL/export/pizza3.csv',index=False)
result4.to_csv('/content/gdrive/My Drive/SQL/export/pizza4.csv',index=False)

link for visualization<br>
https://public.tableau.com/views/Book1_16823748427290/PizzaShopBI?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

#Data source

https://learnbi.online/pizzaproject